In [ ]:
from enum import Enum, IntEnum


class T(IntEnum):
    A = 0


type(T.A)
int(T.A)
T.A == 0

In [ ]:
import os
import pickle
import random
with open("./pickles/cdf_real2.pickle", "rb") as f:
    vlist = pickle.load(f)
vlist

In [ ]:
from glob import glob
folders = ["cropped_faces", "cropped_faces_250", "landmarks", "semantic_videos", "cropped_faces(landmark)"]
files = []
for v in vlist:
    for folder in folders:
        # files.extend(glob(f"/home/od/Desktop/Dataset/DFDC/{folder}/{os.path.split(v)[-1][:-4]}*.*"))
        files.extend(glob(v.replace("videos", folder).replace(".mp4", "*.*")))
files

In [ ]:
len(files), len(vlist)

In [ ]:
for file in files:
    os.remove(file)

In [ ]:
import cv2
import facer
import numpy as np
import torch
from tqdm import tqdm
from matplotlib import pyplot as plt
from torchvision.transforms.functional import normalize
from preprocessing.extract_faces import get_video_clip, save_video_lossless

# video_path = "/home/od/Desktop/Dataset/CelebDF/Fake/cropped_faces/id0_id1_0000.avi"
video_path = "/home/od/Desktop/Dataset/DFDC/videos/nwpjsjarju.mp4"
fps, frames = get_video_clip(video_path, stride=1)

In [ ]:
frame_indices = [i for i in range(0, 300, 1)]

In [ ]:
landmarks = np.load(f"/home/od/Desktop/Dataset/DFDC/test_lm/rfwtnztcsj.npy")

In [ ]:
landmarks.shape

In [ ]:
landmarks.reshape(-1, 2).shape

In [ ]:
# import random
# i = random.randrange(0, len(frame_indices))
# print(i)
# plt.figure(figsize=(25, 15))
# plt.imshow(cv2.cvtColor(frames[frame_indices[i]], cv2.COLOR_BGR2RGB))
# plt.scatter(landmarks[i, :, 0], landmarks[i, :, 1], alpha=0.5, s=1)

In [ ]:
import cv2
import facer
import numpy as np
import torch
from tqdm import tqdm
from matplotlib import pyplot as plt
from torchvision.transforms.functional import normalize
from preprocessing.extract_faces import get_video_clip, save_video_lossless

# video_path = "/home/od/Desktop/Dataset/CelebDF/Fake/cropped_faces/id0_id1_0000.avi"
video_path = "/stock/FaceForensicC23/videos/NT/217_117.mp4"
fps, frames = get_video_clip(video_path, stride=1)

In [ ]:
class FaceData:
    def __init__(self, _lm, _bbox, _idx):
        ema_lm = _lm
        lm = [_lm]
        bbox = [_bbox]
        idx = [_idx]

    def last(self):
        return ema_lm

    def add(self, _lm, _bbox, _idx):
        ema_lm = ema_lm * 0.5 + _lm * 0.5
        lm.append(_lm)
        bbox.append(_bbox)
        idx.append(_idx)

    def __len__(self):
        return len(lm)

In [ ]:
import pickle
with open("./pickles/217_117.pickle", "rb") as f:
    frame_faces = pickle.load(f)

In [ ]:
face_dbs = []
for faces, index in zip(frame_faces, frame_indices):
    frame_landmarks = np.stack([face["landmarks"] for face in faces])
    matched_indices = []

    for face_data in face_dbs:

        lm_diff = np.sum(
            np.linalg.norm(frame_landmarks - face_data.last(), axis=-1),
            axis=1
        ) / frame_landmarks.shape[1]

        if (np.min(lm_diff) > 100):
            continue

        closest_idx = np.argmin(lm_diff)
        matched_indices.append(closest_idx)
        face_data.add(faces[closest_idx]["landmarks"], faces[closest_idx]["bbox"], index)

    for i, face in enumerate(faces):
        if i in matched_indices:
            continue
        else:
            _landmark = face['landmarks']
            _bbox = face['bbox']
            face_dbs.append(FaceData(_landmark, _bbox, index))

In [ ]:
print(len(face_dbs), len(face_dbs[0].lm), len(frames))

In [ ]:
len(sorted(face_dbs, key=lambda x: len(x), reverse=True)[0])

In [ ]:
[i for i in range(0, len(frames)) if not i in face_dbs[0].idx]

In [ ]:
import random
fid = 0
i = random.randrange(0, len(face_dbs[fid]))
i = 210
landmarks = face_dbs[fid].lm[i]
plt.figure(figsize=(25, 15))
plt.imshow(cv2.cvtColor(frames[face_dbs[fid].idx[i]], cv2.COLOR_BGR2RGB))
plt.scatter(landmarks[:, 0], landmarks[:, 1], alpha=0.5, s=5)

In [ ]:
import random
idx = 211
faces = [face["landmarks"] for face in frame_faces[idx]]
print(len(faces))
landmarks = np.stack(faces).reshape((-1, 2))
plt.figure(figsize=(25, 15))
plt.imshow(cv2.cvtColor(frames[idx], cv2.COLOR_BGR2RGB))
plt.scatter(landmarks[:, 0], landmarks[:, 1], alpha=0.5, s=5)

In [ ]:
# Load FaRL weights
import torch
from src.clip import clip
device = "cpu"
model, preprocess = clip.load("ViT-B/16", device=device)
model = model.to(device)
farl_state = torch.load("./misc/FaRL-Base-Patch16-LAIONFace20M-ep16.pth")
model.load_state_dict(farl_state["state_dict"], strict=False)

In [ ]:
# Albumentation Effect Test Site
import cv2
import torchvision
import albumentations as alb
from matplotlib import pyplot as plt
vid_reader = torchvision.io.VideoReader(
    "datasets/ffpp/real/c23/videos/000.avi",
    "video"
)
vid_reader.seek(2.5)
image = next(vid_reader)
del vid_reader

x = alb.ReplayCompose(
    [
        # alb.RandomCropFromBorders(0.5,0.5,0.5,0.5,True)
        alb.RandomResizedCrop(150, 150, scale=(0.5, 0.8), ratio=(1, 1), always_apply=True)
        # alb.RandomCropNearBBox(always_apply=True)
        # alb.RandomScale((-0.5,-0.1),always_apply=True),
        # alb.Resize(150,150)
        # alb.Resize(150,150,always_apply=1)
        # alb.Affine([0.5,1.0],keep_ratio=True,always_apply=True),
        # alb.Affine([1.5,2.0],keep_ratio=True,always_apply=True),
    ],
    p=1.
)

image = image["data"].numpy().transpose((1, 2, 0))
result = x(image=image)
image = result["image"]
print(result["replay"]["transforms"][0]["params"])
# image = cv2.cvtColor(image.transpose((1,2,0)),cv2.COLOR_BGR2RGB)
plt.imshow(image)

In [ ]:

x1 = torch.randn((1, 10))
x2 = torch.randn((1, 10))
v1 = torch.randn((1, 10))
v2 = torch.randn((1, 10))
for i in range(5, 500, 5):
    print("Ratio:", i)
    _x1 = v1 / i
    _x2 = v2 / i
    print(
        torch.mean(torch.nn.functional.cosine_similarity(
            torch.stack((x1 + _x1, x2 + _x2)), torch.stack((x1, x2)), dim=-1)),
        torch.nn.functional.cosine_similarity(x1 + _x1, x2 + _x2, dim=-1),
        torch.nn.functional.cosine_similarity(x1, x2, dim=-1)
    )

In [ ]:
I2 = 0

In [ ]:
import pickle
import torch

with open("misc/ow_semantic_patches.pickle", "rb") as f:
    ow_s = pickle.load(f)
with open("misc/semantic_patches.pickle", "rb") as f:
    s = pickle.load(f)

parts = list(s["k"].keys())
for l in range(12):
    print(f"L:{l}, Parts:{parts}")
    print(
        torch.nn.functional.cosine_similarity(
            torch.stack([s["k"][p][l] for p in parts], dim=0).unsqueeze(0),
            torch.stack([ow_s["k"][p][l] for p in parts], dim=0).unsqueeze(1),
            dim=-1
        )
    )

In [ ]:
import torch
for name, path in [
    ("10", "logs/test/vocal-bee-1132/best_weights.pt"),
    ("20", "logs/test/peachy-haze-1123/best_weights.pt"),
    ("50", "logs/test/olive-water-1118/best_weights.pt")
]:
    print(f"===={name}====")
    prompts = [v for k, v in torch.load(path, "cpu").items() if "prompt" in k][0]
    result = torch.nn.functional.cosine_similarity(prompts.unsqueeze(1), prompts.unsqueeze(2), dim=-1)
    print(result.mean(dim=-1))

In [ ]:
from torch import nn
import math
width = 768
patch_size = 14
scale = width**-0.5
val = math.sqrt(6. / (3 * patch_size**2 + 1) + width)
print(val)
layers = 12
prompts = 10
prompt_embeddings = nn.Parameter(scale * torch.randn(layers, prompts, width))
# nn.init.uniform_(prompt_embeddings.data, -val, val)  # xavier_uniform initialization

torch.nn.functional.cosine_similarity(prompt_embeddings.unsqueeze(
    1), prompt_embeddings.unsqueeze(2), dim=-1).mean(dim=-1)

In [ ]:
import torch
from src.clip import clip
origin = clip.load("ViT-B/16", prompts=0)[0].visual.float().cpu().state_dict()

new = {
    k[8:]: v for k, v in torch.load("logs/test/iconic-durian-1140/best_weights.pt", map_location="cpu").items() if k[:7] == "encoder"
}